In [1]:
import joblib
import json
import pandas as pd
import numpy as np
import xgboost
from sklearn.metrics import mean_absolute_error

In [2]:
def preprocess_data(df, with_target=False):
    
    # видаляємо рядки з пропущеними значеннями 
    df.dropna(axis='rows', inplace=True)
    
    # читаємо словарі з кодуваннями країн, джерел інформації та платформ
    with open(file='country_label_dict.json', mode='r', encoding='utf-8') as file:
        country_label_dict = json.loads(file.read())
    with open(file='media_source_dict.json', mode='r', encoding='utf-8') as file:
        media_source_dict = json.loads(file.read())
    with open(file='platforms_label_dict.json', mode='r', encoding='utf-8') as file:
        platforms_label_dict = json.loads(file.read())
    
    # Label Encoding платформи, коду країни, джерела інформації зі словарів з тренувального набору
    df['platform'] = df['platform'].apply(lambda x: platforms_label_dict[x])
    df['country_code'] = df['country_code'].apply(lambda x: country_label_dict[x])
    df['media_source'] = df['media_source'].apply(lambda x: media_source_dict[x])
    
    # переведемо 'str' дату до datetime64 та отримаємо фічі року, місяця, дня, та дня неділі
    df['install_date'] = pd.to_datetime(df['install_date'], format='%Y-%m-%d')
    df['year'] = df.install_date.dt.year
    df['month'] = df.install_date.dt.month
    df['day'] = df.install_date.dt.day
    df['day_of_week'] = df.install_date.dt.day_of_week
    df.drop(columns=['install_date'], inplace=True)
    
    # пробуємо різну комбінацію фіч
    df['diamond_spent_diff_7_0'] = df['diamonds_spent_day7'] - df['diamonds_spent_day0']
    df['diamond_spent_diff_7_1'] = df['diamonds_spent_day7'] - df['diamonds_spent_day1']
    df['diamond_spent_diff_7_3'] = df['diamonds_spent_day7'] - df['diamonds_spent_day3']
    df['diamond_spent_ratio_7_0'] = df['diamonds_spent_day7'] / df['diamonds_spent_day0']
    df['diamond_spent_ratio_7_1'] = df['diamonds_spent_day7'] / df['diamonds_spent_day1']
    df['diamond_spent_ratio_7_3'] = df['diamonds_spent_day7'] / df['diamonds_spent_day3']
    df['diamond_spent_sum_7_3'] = df['diamonds_spent_day7'] + df['diamonds_spent_day3']
    df['diamond_spent_diff_3_0'] = df['diamonds_spent_day3'] - df['diamonds_spent_day0']
    df['diamond_spent_diff_3_1'] = df['diamonds_spent_day3'] - df['diamonds_spent_day1']
    df['diamond_spent_ratio_3_0'] = df['diamonds_spent_day3'] / df['diamonds_spent_day0']
    df['diamond_spent_ratio_3_1'] = df['diamonds_spent_day3'] / df['diamonds_spent_day1']
    df['diamond_spent_diff_1_0'] = df['diamonds_spent_day1'] - df['diamonds_spent_day0']
    df['diamond_spent_ratio_1_0'] = df['diamonds_spent_day1'] / df['diamonds_spent_day0']
    df['target_full_day3'] = df['app_sub_ltv_day3'] + df['app_iap_ltv_day3'] + df['ad_ltv_day3']
    df['target_iap_sub_ratio'] = df['app_iap_ltv_day3'] / df['app_sub_ltv_day3']
    df['target_sub_ltv_ratio'] = df['app_sub_ltv_day3'] / df['ad_ltv_day3']
    
    df['tickets_spent_diff_7_3'] = df['tickets_spent_day7'] - df['tickets_spent_day3']
    df['tickets_spent_ratio_7_3'] = df['tickets_spent_day7'] / df['tickets_spent_day3']
    
    df['app_iap_ltv_diamonds_spent_ratio_3'] = df['app_iap_ltv_day3'] / df['diamonds_spent_day3']

    df['diamonds_spent_app_iap_ltv_ratio_0'] = df['diamonds_spent_day0'] / df['app_iap_ltv_day0']
    
    df['diamond_spent_tickets_spent_ratio_7_3'] = df['tickets_spent_ratio_7_3'] / df['diamond_spent_ratio_7_3']
    
    df['app_iap_ltv_diff_1_3'] = df['app_iap_ltv_day1'] - df['app_iap_ltv_day3']
    df['app_iap_ltv_diff_0_3'] = df['app_iap_ltv_day0'] - df['app_iap_ltv_day3'] 
    
    income_columns = ['app_sub_ltv_day0', 'app_sub_ltv_day1', 'app_sub_ltv_day3', 
                  'app_iap_ltv_day0', 'app_iap_ltv_day1', 'app_iap_ltv_day3']

    df['is_profitable'] = (df[income_columns].sum(axis=1) > 0).astype(np.int32)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.fillna(0)
    
    # оптимізуємо типи даних 
    float32_cols = [c for c in df if df[c].dtype == 'float64']
    int32_cols = [c for c in df if df[c].dtype in ['int64', 'int16', 'int8']]

    df[float32_cols] = df[float32_cols].astype(np.float32)
    df[int32_cols] = df[int32_cols].astype(np.int32)
    
    # якщо у датасеті є target значення, то ми відділяємо їх 
    if with_target:
        Y = df['target_full_ltv_day30']
        df.drop(columns=['target_sub_ltv_day30', 'target_iap_ltv_day30', 'target_ad_ltv_day30', 'target_full_ltv_day30'], inplace=True)
    
    # завантажуємо StandardScaler з тренувального набору
    scaler = joblib.load("scaler_data.sav")
    df = scaler.transform(df)
    
    if with_target:
        return df, Y
    else:
        return df

In [3]:
# флаг відповідає за наявність target фіч у датасеті
with_target = True
# шлях до файлу з даними
path_to_csv = 'TEST TASK (dataset).csv'
df = pd.read_csv(path_to_csv)

In [4]:
# робимо препроцессінг датасету
if with_target:
    df, Y = preprocess_data(df, with_target)
else:
    df = preprocess_data(df, with_target)

In [5]:
# Імпортуємо модель
model = joblib.load("model.sav")

In [6]:
if with_target:
    print('MAE:', mean_absolute_error(Y, model.predict(xgboost.DMatrix(df))))
else:
    predict = model.predict(xgboost.DMatrix(df))

MAE: 0.066657394


### В якості основної метрики було обратно MAE. Чому саме MAE?

1. Так як цільова метрика має деяку періодичність, завдяки чому деякі дані представляють собою викиди у деякій формі. Тобто якщо в грі наприклад виходить якесь оновлення пов'язане з контентом, як наприклад нові глави, то активна кількість людей відразу після оновлення буде більшою, ніж кількість людей зазвичай до цього оновлення (або не обов'язково з контентом, таке оновлення, яке приносить з собою додаткову можливість монетизації гри, як наприклад додатковий сюжет, косметика і т.д). А якщо кількість одночасних гравців більше, то буде і більший прибуток, який вище, ніж зазвичай до оновлення гри. Таким чином модель з метрикою MAE буде націлена на прогноз саме медіани, а в порівнянні наприклад метрика RMSE буде націлена саме на прогноз середнього значення. Тому і важливіше наближувати прогноз до медіани, адже в загальному випадку медіана значно нижча за середне (з EDA можна зробити висновок, що цільова фіча target_full_ltv_day30 має медіану 0 та середнє ~0.26)

2. Метрика MAE гарна для того, щоб розділити наш набір на 2 частини. Про які частини йде річ? Справа в тому, що з EDA можна побачити, що тільки ~30.6% всіх гравців принесли якийсь прибуток. Тобто можна чітко сказати, що є 2 класи гравців: які не приносять жодного прибутку та ті, хто приносить якийсь прибуток.

3. Метрика MAE більш робастна до викидів. З EDA можна помітити деякі аномалії, як наприклад те що у якихось гравців аномально висока кількість сессій, як наприклад у деяких гравців за 0 день 100+ сесій, що є ~0.1% від всього набору даних. Також є випадки, коли у деяких гравців є більше 15.000 кристалів за умови, що вони не принесли ні одного центу прибутку та мають маленьку кількість закінчених / закритих / відкритих глав, що каже про те, що вони не могли отримати таку велику кількість кристалів просто граючи в гру без донату. Оскільки ми не маємо точної інформації звідки беруться такі дані, можна припустити, що можуть бути деякі баги та інші види помилок, що формують ці викиди. І саме метрика MAE найкраще справляється з цим, наприклад в порівнянні з RMSE, яка дуже чутлива до викидів та одна велика помилка може коштувати великій кількості оцінки RMSE. Саме через це ми майже не займалися видаленням викидів з датасету, адже MAE добре справляється з ними.

Таким чином в приведених прикладах можна зрозуміти чому саме MAE, а не RMSE. А що стосовно MAPE?
Оцінка MAPE дещо спотворена, так як ця метрика ділить кожну помилку окремо на цільове значення. Великі помилки в момент маленького цільового значення сильно впливають на цю оцінку, через це після оптимізації цієї метрики ми отримаємо дивний прогноз, який у більшості випадків буде меншим за цільове значення. Тобто MAPE просто дає низький прогноз через те, що надає велику вагу помилкам коли цільова метрика маленька. Тому ми взагалі не використовували цю метрику та не орієнтувалися на неї через її особливості, які не підходять для нашої задачі.

Але ми привели оцінки цих 3 метрик для нашої моделі у тренувальному ноутбуці